## 🏥 Étude de cas : Analyse du parcours patient et prédiction des résultats médicaux

🎯 **Objectif** : Étudier les facteurs influençant les coûts de soins, la durée d’hospitalisation et les résultats médicaux (`Test Results`) afin d’identifier des tendances pertinentes dans la gestion hospitalière.

Le dataset contient des données synthétiques de patients hospitalisés, incluant des informations personnelles, cliniques, financières et administratives.


In [1]:
# 📦 Import des bibliothèques essentielles
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, f_oneway
import warnings
warnings.filterwarnings("ignore")

# 🎯 Chargement du dataset
df = pd.read_csv("healthcare_dataset.csv")

# 🧹 Nettoyage initial (si besoin)
df.columns = df.columns.str.strip()  # Suppression des espaces en trop dans les noms de colonnes

# 🕒 Conversion des dates
df["Date of Admission"] = pd.to_datetime(df["Date of Admission"])
df["Discharge Date"] = pd.to_datetime(df["Discharge Date"])

# 🧮 Création d'une variable dérivée : durée d'hospitalisation
df["Hospital Stay (Days)"] = (df["Discharge Date"] - df["Date of Admission"]).dt.days

# 🔎 Aperçu rapide
df.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results,Hospital Stay (Days)
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal,2
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive,6
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal,15
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal,30
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal,20


## 🟨 Partie 1 – Statistiques univariées : `Billing Amount`

### 📘 Notion
Analyse descriptive d’une variable quantitative (`Billing Amount`) à l’aide d’indicateurs classiques :
- Moyenne, médiane, mode
- Variance, écart-type
- Intervalle interquartile (IQR)
- Étendue (écart entre valeur max et min)

### 🎯 Objectif dans l’étude
Le `Billing Amount` correspond au montant total facturé à un patient.  
L’objectif ici est de :
- Résumer la **distribution des coûts**,
- Identifier **la variabilité**,
- Détecter d’éventuelles **valeurs extrêmes**,
- Préparer les futures comparaisons par groupe (âge, genre, type d’admission...).



### 🧮 Formules mathématiques utilisées

$$
\bar{x} = \frac{1}{n} \sum_{i=1}^{n} x_i \quad \text{(Moyenne)}
$$

$$
s^2 = \frac{1}{n - 1} \sum_{i=1}^{n}(x_i - \bar{x})^2 \quad \text{(Variance)}
$$

$$
s = \sqrt{s^2} \quad \text{(Écart\text{-}type)}
$$

$$
\text{IQR} = Q_3 - Q_1
$$

$$
\text{Étendue} = x_{\text{max}} - x_{\text{min}}
$$


In [4]:
# 📊 Statistiques descriptives sur Billing Amount
billing_amount = df["Billing Amount"].dropna()

billing_stats = {
    "Moyenne": round(billing_amount.mean(), 2),
    "Médiane": round(billing_amount.median(), 2),
    "Mode": round(billing_amount.mode()[0], 2),
    "Variance": round(billing_amount.var(), 2),
    "Écart-type": round(billing_amount.std(), 2),
    "Minimum": round(billing_amount.min(), 2),
    "Maximum": round(billing_amount.max(), 2),
    "IQR": round(billing_amount.quantile(0.75) - billing_amount.quantile(0.25), 2)
}

billing_stats


{'Moyenne': np.float64(25539.32),
 'Médiane': np.float64(25538.07),
 'Mode': np.float64(-1316.62),
 'Variance': np.float64(201965437.04),
 'Écart-type': np.float64(14211.45),
 'Minimum': np.float64(-2008.49),
 'Maximum': np.float64(52764.28),
 'IQR': np.float64(24579.28)}

### 📊 Interprétation

- **Moyenne** ≈ 25 539 € et **médiane** ≈ 25 538 € → les deux sont presque identiques, ce qui indique une **distribution relativement symétrique** du `Billing Amount`.

- **Écart-type** ≈ 14 211 € → cela montre une **grande dispersion** des montants facturés. Les patients ne paient donc pas tous le même niveau de soin : certaines factures sont bien plus élevées que d'autres.

- **Variance** très élevée (~ 20 millions) → accentue l'idée d'une **variabilité importante**, attendue pour des données monétaires.

- **IQR** ≈ 24 579 € → les 50 % des valeurs centrales sont dispersées sur un intervalle très large, ce qui montre une **hétérogénéité significative** entre les patients.

- **Maximum** = 52 764 € et **minimum** = -2 008 € → la valeur négative est **incohérente** dans ce contexte (on ne rembourse pas au patient), ce qui suggère soit une **valeur aberrante**, soit une **erreur de saisie**. À corriger ou supprimer en Partie 5.

- **Mode** = -1 316 € → invraisemblable, probablement dû à une **anomalie ou à un regroupement de valeurs aberrantes**.

📌 **Conclusion** :
- La variable `Billing Amount` présente une **très forte dispersion** et probablement des **valeurs anormales**.
- Ces éléments justifient une **analyse complémentaire sur les valeurs extrêmes** (Partie 5) ainsi que des **comparaisons par groupe** pour identifier des causes structurelles.


## 🟩 Partie 2 – Corrélation entre `Age` et `Billing Amount`

### 📘 Notion
Analyse bivariée entre deux variables quantitatives : **âge du patient** et **montant facturé**.  
On utilise le **coefficient de corrélation de Pearson (r)** pour mesurer la **force et la direction** de la relation linéaire entre ces deux variables.

---

### 🎯 Objectif dans l’étude
Cette analyse permet de :
- Identifier si l’**âge** a un impact sur le **coût des soins**,
- Vérifier s’il existe une **tendance** (ex. : plus âgé = plus coûteux ?),
- Aider à expliquer les **facteurs prédictifs des coûts** dans les parties futures (ex. : modélisation).


### 🧮 Formule de la corrélation de Pearson

$$
r = \frac{\sum_{i=1}^{n} (x_i - \bar{x})(y_i - \bar{y})}
         {\sqrt{\sum_{i=1}^{n} (x_i - \bar{x})^2 \cdot \sum_{i=1}^{n} (y_i - \bar{y})^2}}
$$

où :
- \( x_i \) : les âges des patients
- \( y_i \) : les montants facturés
- \( r \in [-1, 1] \) : coefficient de corrélation linéaire


In [5]:
# 📊 Corrélation entre Age et Billing Amount
correlation_df = df[["Age", "Billing Amount"]].dropna()
correlation = correlation_df.corr().loc["Age", "Billing Amount"]
correlation


np.float64(-0.0038319421186176848)

### 📊 Interprétation du résultat

- Le coefficient de corrélation \( r \approx -0.0038 \) indique une **quasi-absence de lien linéaire** entre l’âge des patients et le montant facturé (`Billing Amount`).

- Cette valeur est extrêmement proche de zéro, ce qui signifie que **l'âge ne permet pas d'expliquer la variation du coût des soins** dans ce dataset.

- Il n’y a **ni tendance croissante ni décroissante** claire : des patients jeunes peuvent être très coûteux, tout comme des patients âgés peuvent ne pas l’être du tout.

- Cela peut s’expliquer par le fait que le **coût dépend davantage de la pathologie**, de la durée d'hospitalisation, ou du type d’admission plutôt que de l’âge seul.

📌 **Conclusion** :
- Cette variable ne sera **pas utile comme prédicteur principal** du coût dans une modélisation linéaire simple.
- Elle reste néanmoins intéressante à croiser **avec d'autres variables**, comme le `Medical Condition` ou le `Type d’admission`.
